<a href="https://colab.research.google.com/github/Nourbh17/PFA-SC-FEGAN-Face-Editing/blob/main/losses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

class GeneratorLosses():
    # gsn_loss function computes this generator loss. By negating the mean of the discriminator's output for the fake data,
    # the generator is encouraged to produce data that the discriminator will classify as real,
    # thereby minimizing the Wasserstein distance between the real and fake data distributions.
    def gsn_loss(dis_fake):
        gen_loss = - tf.reduce_mean(dis_fake)
        return gen_loss
    def per_pixel_loss(gen_output, ground_truth, mask, alpha):
        _, n_h, n_w, n_c = ground_truth.get_shape().as_list()

        term_1 = tf.reduce_sum(mask * (gen_output - ground_truth)) / (n_h * n_w * n_c)
        term_2 = tf.reduce_sum((1. - mask) * (gen_output - ground_truth)) / (n_h * n_w * n_c)

        loss = term_1 + alpha * term_2
        return loss

    pool_layers = ["pool1", "pool2", "pool3"]
    def get_vgg16_layers(layers_names):
        vgg = VGG16(include_top=False, input_shape=(512, 512, 3), weights='imagenet')
        vgg.trainable = False
        outputs = [vgg.get_layer(name).output for name in layers_names]
        model = Model(inputs=vgg.input, outputs=outputs)
        return model
    vgg = get_vgg16_layers(pool_layers)

    def perceptual_loss(complete_image, ground_truth, gen_image):
        features_ground_truth = vgg(ground_truth)
        features_output_image = vgg(complete_image)
        features_gen_image = vgg(gen_image)


        assert len(features_ground_truth) == 3
        assert len(features_output_image) == 3
        assert len(features_gen_image) == 3

        term_1 = 0.
        term_2 = 0.
        for i in range(len(features_ground_truth)):
            _, n_h, n_w, n_c = features_ground_truth[i].get_shape().as_list()
            loss = tf.reduce_sum(features_output_image[i] - features_ground_truth[i]) / (n_h * n_w * n_c)
            term_1 += loss
            loss2 = tf.reduce_sum(features_gen_image[i] - features_ground_truth[i]) / (n_h * n_w * n_c)
            term_2 += loss2
        return term_1 + term_2


